In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm
import PyPDF2
import re
from os import listdir
from os.path import isfile, join

## URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)

In [2]:
df_vs=pd.read_csv("KRFrames/df_v_Hauptseiten")

In [3]:
df_vs

,Unnamed: 0,index,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,3020,/gast/geschaefte/2008,Postulat von Luzian Franzini und Esther Haas b...,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,1,1,3019,/gast/geschaefte/2007,"Motion von Luzian Franzini, Rita Hofer, Tabea ...",Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
2,2,2,3018,/gast/geschaefte/2006,"Interpellation von Fabio Iten, Laura Dittli un...",Überweisung an Regierungsrat,Volkswirtschaftsdirektion,Interpellation,hängig
3,3,3,3017,/gast/geschaefte/2005,"Postulat von Tabea Zimmermann Gibson, Stéphani...",Überweisung an Regierungsrat,Direktion des Innern,Postulat,hängig
4,4,6,3014,/gast/geschaefte/2002,Interpellation der SVP-Fraktion betreffend mob...,Kenntnisnahme,Sicherheitsdirektion,Interpellation,hängig
...,...,...,...,...,...,...,...,...,...
1215,1215,1946,762,/gast/geschaefte/1331,Motion der erweiterten Justizprüfungskommissio...,Datenmigration,NaN,Motion,abgeschlossen
1216,1216,1952,666,/gast/geschaefte/1325,Motion der Kommission Teilrevision Personalges...,Datenmigration,NaN,Motion,abgeschlossen
1217,1217,1968,304,/gast/geschaefte/1309,Motion von Manuela Weichelt betreffend HIV-Prä...,Datenmigration,NaN,Motion,abgeschlossen
1218,1218,1969,282,/gast/geschaefte/1712,Motion von Christoph Hohler betreffend Radstre...,Frist erstreckt,Baudirektion,Motion,hängig


# hier ein Test von Barnaby, wie man die Namen alleine mit "Split" separieren kann

- Vielleicht zuerst noch mit Pandas nur die Vorstösse raussuchen. Ist ja das, was mich interessiert.
- Rausfinden, wie ich die Namen nun rausziehe und in ein aufklappbares Dataframe stelle


In [4]:

def splitting(elem):
    elem = elem.split('betreffend')[0].strip()
    if 'von' in elem:
        elem = elem.split('von')[1].strip()
        elem = elem.replace(' und', ',')
    elif 'der' in elem:
        elem = elem.split('der')[1].strip()    
    return elem
df_vs["Geschäft"]=df_vs['Geschäft'].apply(splitting) #.apply ist um eine Funktion auf eine ganze Reihe Pandas einzusetze

# Nun rausfinden, wie ich ein DataFrame machen kann, das man aufklappt

In [5]:
df_vs.head()

,Unnamed: 0,index,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,3020,/gast/geschaefte/2008,"Luzian Franzini, Esther Haas",Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,1,1,3019,/gast/geschaefte/2007,"Luzian Franzini, Rita Hofer, Tabea Zimmermann ...",Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
2,2,2,3018,/gast/geschaefte/2006,"Fabio Iten, Laura Dittli, Anna Bieri",Überweisung an Regierungsrat,Volkswirtschaftsdirektion,Interpellation,hängig
3,3,3,3017,/gast/geschaefte/2005,"Tabea Zimmermann Gibson, Stéphanie Vuichard, E...",Überweisung an Regierungsrat,Direktion des Innern,Postulat,hängig
4,4,6,3014,/gast/geschaefte/2002,SVP-Fraktion,Kenntnisnahme,Sicherheitsdirektion,Interpellation,hängig


In [6]:
#das funktionerit also nicht
def splitkomma(elem):
    elem=elem.split(", ")
    return elem


In [7]:
df_vs["Geschäft"]=df_vs["Geschäft"].apply(splitkomma) 

In [39]:
# Hier die Lösung, wie ich für jeden Namen ein einzelnes Geschäft erhalte

In [8]:
df_ex=df_vs.explode('Geschäft').reset_index(drop=True)

In [9]:
df_ex.rename(columns={'Geschäft':'Namenganz'}, inplace=True)

In [11]:
df_ex.to_csv("KRFrames/df_ex")

In [12]:
df_ex

,Unnamed: 0,index,VorlageNR,url,Namenganz,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,3020,/gast/geschaefte/2008,Luzian Franzini,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,0,0,3020,/gast/geschaefte/2008,Esther Haas,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
2,1,1,3019,/gast/geschaefte/2007,Luzian Franzini,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
3,1,1,3019,/gast/geschaefte/2007,Rita Hofer,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
4,1,1,3019,/gast/geschaefte/2007,Tabea Zimmermann Gibson,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
...,...,...,...,...,...,...,...,...,...
1942,1215,1946,762,/gast/geschaefte/1331,erweiterten Justizprüfungskommission,Datenmigration,NaN,Motion,abgeschlossen
1943,1216,1952,666,/gast/geschaefte/1325,Kommission Teilrevision Personalgesetz,Datenmigration,NaN,Motion,abgeschlossen
1944,1217,1968,304,/gast/geschaefte/1309,Manuela Weichelt,Datenmigration,NaN,Motion,abgeschlossen
1945,1218,1969,282,/gast/geschaefte/1712,Christoph Hohler,Frist erstreckt,Baudirektion,Motion,hängig


In [26]:
df_ex[df_ex['Namenganz'].str.contains("Langenegger")]


,Unnamed: 0,VorlageNR,url,Namenganz,Stand,Zuständigkeit,Art des Geschäfts,Status
1306,791,1761,/gast/geschaefte/54,Beni Langenegger,Datenmigration,NaN,Interpellation,abgeschlossen
1656,1016,1363,/gast/geschaefte/954,Beni Langenegger,Datenmigration,NaN,Interpellation,abgeschlossen


### Problem erkannt: 
offenbar werden gekoppelte Namen in den Geschäften nur mit den ersten aufgeführt.

In [19]:
df_ex.set_index("Unnamed: 0")


,index,VorlageNR,url,Namenganz,Stand,Zuständigkeit,Art des Geschäfts,Status
Unnamed: 0,,,,,,,,
0,0,3020,/gast/geschaefte/2008,Luzian Franzini,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
0,0,3020,/gast/geschaefte/2008,Esther Haas,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,1,3019,/gast/geschaefte/2007,Luzian Franzini,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
1,1,3019,/gast/geschaefte/2007,Rita Hofer,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
1,1,3019,/gast/geschaefte/2007,Tabea Zimmermann Gibson,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
...,...,...,...,...,...,...,...,...
1215,1946,762,/gast/geschaefte/1331,erweiterten Justizprüfungskommission,Datenmigration,NaN,Motion,abgeschlossen
1216,1952,666,/gast/geschaefte/1325,Kommission Teilrevision Personalgesetz,Datenmigration,NaN,Motion,abgeschlossen
1217,1968,304,/gast/geschaefte/1309,Manuela Weichelt,Datenmigration,NaN,Motion,abgeschlossen


In [13]:
del df_ex["index"]

In [14]:
# Nun das Frame mit den Namen einlesen
df_np=pd.read_csv('KRFrames/Alle_Namen_mit_Allianz_und_ohne')

In [15]:
#von der Bearbeitung hat es nun noch einen anderen Index. stört uns nicht - Wir entfernen "unnamed:0"
del df_np["Unnamed: 0"]

In [16]:
df_np #voila

,Namenganz,P
0,Judith Wild,FDP
1,Madeleine Landolt,SGA
2,Peter Bossard,FDP
3,Susi Frei,FDP
4,Alois Henggeler,CVP
...,...,...
386,Iris Hess-Brauer,CVP
387,Mariann Hess-Witschi,Kt.
388,Magda Feldmann-Müller,CSP
389,Sepp Grob-Bieri,CVP


In [19]:
#Nun die beiden zusammenfügen - an den Namen
df_alles=pd.merge(df_np, df_ex, how="right",on= "Namenganz", )

In [37]:
df_alles=df_alles.sort_values("Unnamed: 0")

In [25]:
df_alles.tail(80)#Wieso haben die Kantonsräte noch keine Partei. Nachschauen im Ämterverzeichnis.

,Namenganz,P,Unnamed: 0,VorlageNR,url,Stand,Zuständigkeit,Art des Geschäfts,Status
1894,FDP-Fraktion für eine jährliche Anpassung,NaN,776,1780,/gast/geschaefte/67,Datenmigration,NaN,Motion,abgeschlossen
1895,Beni Langenegger,NaN,791,1761,/gast/geschaefte/54,Datenmigration,NaN,Interpellation,abgeschlossen
1896,Beni Langenegger,NaN,1016,1363,/gast/geschaefte/954,Datenmigration,NaN,Interpellation,abgeschlossen
1897,Karin Julia Stadlin,NaN,794,1758,/gast/geschaefte/52,Datenmigration,NaN,Interpellation,abgeschlossen
1898,Andreas Huwyler,NaN,804,1743,/gast/geschaefte/45,Datenmigration,NaN,Interpellation,abgeschlossen
...,...,...,...,...,...,...,...,...,...
1969,Käty Hofer Buser,NaN,1210,836,/gast/geschaefte/1339,Datenmigration,NaN,Interpellation,abgeschlossen
1970,Kommission Parlamentsreform,NaN,1211,823,/gast/geschaefte/1337,Datenmigration,NaN,Motion,abgeschlossen
1971,Kommission Parlamentsreform,NaN,1212,822,/gast/geschaefte/1336,Datenmigration,NaN,Motion,abgeschlossen
1972,Kommission Teilrevision Personalgesetz,NaN,1216,666,/gast/geschaefte/1325,Datenmigration,NaN,Motion,abgeschlossen


In [26]:
df_alles.set_index("Unnamed: 0")

,Namenganz,P,VorlageNR,url,Stand,Zuständigkeit,Art des Geschäfts,Status
Unnamed: 0,,,,,,,,
0,Esther Haas,ALG,3020,/gast/geschaefte/2008,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,Tabea Zimmermann Gibson,Kt.,3019,/gast/geschaefte/2007,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
1,Andreas Lustenberger,ALG,3019,/gast/geschaefte/2007,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
1,Rita Hofer,ALG,3019,/gast/geschaefte/2007,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
1,Hanni Schriber-Neiger,ALG,3019,/gast/geschaefte/2007,Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
...,...,...,...,...,...,...,...,...
1209,Hans Durrer,SVP,857,/gast/geschaefte/1340,Datenmigration,NaN,Motion,abgeschlossen
1209,Karl Betschart,SVP,857,/gast/geschaefte/1340,Datenmigration,NaN,Motion,abgeschlossen
1213,Marcel Meyer,CVP,810,/gast/geschaefte/1335,Datenmigration,NaN,Motion,abgeschlossen


In [27]:
# nun das df_alles noch abspeichern für den Endcode
df_alles.to_csv("KRFrames/Frame_mit_Namen_und_Parteien")

# IRRWEGE

In [ ]:

nameliste=df_vs["Geschäft"].tolist()

In [ ]:
nameliste

nameliste[1] range(0:1219)

In [ ]:
len(nameliste)

In [ ]:
lestlist=[]
lestlist.append("suppe")
len(lestlist)

In [ ]:
namenliste_n=[]
counter = 0
zaehler = 0

for alleeintraege in df_vs["Geschäft"]: #hier gehe ich jede Liste im Dataframe an.
    print(alleeintraege)
    name = alleeintraege[zaehler] #ich will sie auslesen und die Namen einzeln.

    if zaehler < len(alleeintraege)-1: #da die Listen unterschiedlich lang sind, versuche ich hier dies zu berücksichtigen
        zaehler=zaehler+1 #es scheint leider nicht zu funktionieren. 
    elif zaehler == (alleeintraege)-1: 
        continue
   
          
    
    minidictnamen={"Name":name}
    namenliste_n.append(minidictnamen)
  

In [ ]:
pd.DataFrame(namenliste_n)

In [ ]:
return tuple(namenliste_n)

In [ ]:
#Ich versuche es mal schriftlich bevor ich den Loop schreibe:
liste=[]
namenliste={"sophie", "anna","julia"}
minidicttest={"ART": "Motion", "NUMMER": 23455, "NAMEN":namenliste}
liste.append(minidicttest)

In [ ]:
df_test=pd.DataFrame(liste)

In [ ]:
df_test

In [ ]:
for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value